In [1]:
import json
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.stats import linregress

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.decomposition import PCA


## Extract data

In [2]:
files = glob.glob(r'C:\Users\trist\OneDrive\Documents\game_data\valorant_raw/*.json')
len(files)

3109

In [3]:
with open(files[0], 'r') as f:
    json_data = json.load(f)
json_data = json.loads(json_data)
json_data['data'].keys()

dict_keys(['attributes', 'metadata', 'segments', 'expiryDate'])

In [4]:
files[0]

'C:\\Users\\trist\\OneDrive\\Documents\\game_data\\valorant_raw\\0001be30-1ef6-40d8-8f92-13ffbd093cf1.json'

In [5]:
json_data['data']['attributes']

{'id': '0001be30-1ef6-40d8-8f92-13ffbd093cf1'}

In [6]:
json_data['data']['metadata']

{'modeKey': 'bomb',
 'modeName': 'Normal',
 'modeImageUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/icons/modes/normal.png',
 'modeMaxRounds': 25,
 'duration': 2664924,
 'dateStarted': '2022-07-08T06:53:09.588+00:00',
 'rounds': 24,
 'isRanked': False,
 'queueId': 'unrated',
 'map': '2fb9a4fd-47b8-4e7d-a969-74b4046ebd53',
 'mapName': 'Breeze',
 'mapImageUrl': 'https://titles.trackercdn.com/valorant-api/maps/2fb9a4fd-47b8-4e7d-a969-74b4046ebd53/splash.png'}

In [7]:
game_metadata = json_data['data']['metadata']

In [8]:
json_data['data']['expiryDate']

'2022-08-08T18:11:36.1374295+00:00'

In [9]:
all_segments_types = set()

for i in json_data['data']['segments']:
    all_segments_types.add(i['type'])

all_segments_types

{'player-round',
 'player-round-damage',
 'player-round-kills',
 'player-summary',
 'round-summary',
 'team-summary'}

In [10]:
player_rounds = [i for i in json_data['data']['segments'] if i['type'] == 'player-round']
player_rounds_damage = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-damage']
player_rounds_kills = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-kills']
player_summary = [i for i in json_data['data']['segments'] if i['type'] == 'player-summary']
round_summary = [i for i in json_data['data']['segments'] if i['type'] == 'round-summary']
team_summary = [i for i in json_data['data']['segments'] if i['type'] == 'team-summary']

## player_rounds_kills

In [11]:
player_rounds_kills[0].keys()

dict_keys(['type', 'attributes', 'metadata', 'expiryDate', 'stats'])

In [12]:



player_rounds_kills[0]['attributes']

{'round': 1,
 'platformSlug': 'riot',
 'platformUserIdentifier': 'rimo#SEXY',
 'opponentPlatformSlug': 'riot',
 'opponentPlatformUserIdentifier': 'Hennessyy#AKA47'}

In [13]:
player_rounds_kills[0]['metadata']

{'platformInfo': {'platformSlug': 'riot',
  'platformUserId': None,
  'platformUserHandle': 'rimo#SEXY',
  'platformUserIdentifier': 'rimo#SEXY',
  'avatarUrl': None,
  'additionalParameters': None},
 'opponentPlatformInfo': {'platformSlug': 'riot',
  'platformUserId': None,
  'platformUserHandle': 'Hennessyy#AKA47',
  'platformUserIdentifier': 'Hennessyy#AKA47',
  'avatarUrl': None,
  'additionalParameters': None},
 'opponentLocation': {'x': 4917, 'y': 3223},
 'playerLocations': [{'puuid': 'gaxPhPH2L0l07DbnPvzTzKcQzNi3qJfuNjigpnUV3UBkRNdDpGdhyhbU2KWVV1QbHIAXZrowoPOkcA',
   'viewRadians': 2.4492486,
   'location': {'x': 5927, 'y': 4912}},
  {'puuid': '-2GqRI2DorSneRUZgv2T47wOha66l9v3qUbG7Pxw1VibAdJkrHnLI6WgCZeAj3O8HUtAuAyuD4suLQ',
   'viewRadians': 5.6062036,
   'location': {'x': 4806, 'y': 5496}},
  {'puuid': 'LyIVoyMdyMEmaLGGwwI7eyO0vc6UIlwreIZF7OaidgY0SyLa4vF3Q6jGLlB_KBtCkhHfMtTNYaWi1A',
   'viewRadians': 5.0044217,
   'location': {'x': 3969, 'y': 6342}},
  {'puuid': 'q8cbMuUjAnP0Hj

In [14]:
player_rounds_kills[0]['stats']

{'damage': {'rank': None,
  'percentile': None,
  'displayName': None,
  'displayCategory': None,
  'category': None,
  'metadata': {},
  'value': 100,
  'displayValue': '100',
  'displayType': 'Number'}}

In [15]:
files = glob.glob(r'C:\Users\trist\OneDrive\Documents\game_data\valorant_raw/*.json')
print(len(files))
kill_records = list()

for file in files:
    with open(file, 'r') as f:
        json_data = json.load(f)
    json_data = json.loads(json_data)
    player_rounds_kills = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-kills']
    for kill in player_rounds_kills:        
        kill_records.append({'weaponName':kill['metadata']['weaponName'],
                            'round':kill['attributes']['round'],
                            'roundTime':kill['metadata']['roundTime']})
kill_df = pd.DataFrame.from_dict(kill_records)
kill_df.head()

3109


,weaponName,round,roundTime
0,Classic,1,20725
1,Ghost,1,37182
2,Classic,1,51285
3,Classic,1,23532
4,Classic,1,27015


In [16]:
kill_df['weaponName'].value_counts().index.tolist()

['Vandal',
 'Phantom',
 'Spectre',
 'Ghost',
 'Classic',
 'Operator',
 'Sheriff',
 'Guardian',
 'Marshal',
 'Odin',
 'Judge',
 'Bulldog',
 'Stinger',
 'Frenzy',
 'Ares',
 'Shorty',
 'Bucky']

## round_summary

In [17]:
round_summary[19]

{'type': 'round-summary',
 'attributes': {'round': 20},
 'metadata': {},
 'expiryDate': '0001-01-01T00:00:00+00:00',
 'stats': {'roundResult': {'rank': None,
   'percentile': None,
   'displayName': None,
   'displayCategory': None,
   'category': None,
   'metadata': {},
   'value': 'Detonate',
   'displayValue': 'Detonate',
   'displayType': 'String'},
  'winningTeam': {'rank': None,
   'percentile': None,
   'displayName': None,
   'displayCategory': None,
   'category': None,
   'metadata': {},
   'value': 'Blue',
   'displayValue': 'Blue',
   'displayType': 'String'}}}

## player_rounds

In [18]:
player_rounds[17]

{'type': 'player-round',
 'attributes': {'round': 2,
  'platformSlug': 'riot',
  'platformUserIdentifier': 'Corgi Tree#NA1'},
 'metadata': {'teamId': 'Red',
  'agentKey': '707eab51-4836-f488-046a-cda6bf494859',
  'platformInfo': {'platformSlug': 'riot',
   'platformUserId': None,
   'platformUserHandle': 'Corgi Tree#NA1',
   'platformUserIdentifier': 'Corgi Tree#NA1',
   'avatarUrl': None,
   'additionalParameters': None},
  'hasWon': False},
 'expiryDate': '0001-01-01T00:00:00+00:00',
 'stats': {'score': {'rank': None,
   'percentile': None,
   'displayName': 'Score',
   'displayCategory': None,
   'category': None,
   'metadata': {},
   'value': 300,
   'displayValue': '300',
   'displayType': 'Number'},
  'kills': {'rank': None,
   'percentile': None,
   'displayName': 'Kills',
   'displayCategory': None,
   'category': None,
   'metadata': {},
   'value': 1,
   'displayValue': '1',
   'displayType': 'Number'},
  'deaths': {'rank': None,
   'percentile': None,
   'displayName': 'Dea

## round_summary

In [19]:
len(round_summary)

24

In [20]:
# round_summary[22]


In [21]:
def get_winner(round_summary):
    return round_summary[-1]['stats']['winningTeam']['value']

get_winner(round_summary)

'Red'

## player_summary

In [22]:
len(player_summary)

10

In [23]:
player_summary[0].keys()

dict_keys(['type', 'attributes', 'metadata', 'expiryDate', 'stats'])

In [24]:
player_summary[0]['attributes']

{'platformSlug': 'riot', 'platformUserIdentifier': 'Hennessyy#AKA47'}

In [25]:
player_summary[0]['metadata']

{'partyId': '4155c5e2-bf8d-4736-acb1-8ec95e2576b1',
 'teamId': 'Blue',
 'agentKey': 'add6443a-41bd-e414-f6ad-e58d267f4e95',
 'agentName': 'Jett',
 'agentColor': '#98DCFF',
 'agentImageUrl': 'https://titles.trackercdn.com/valorant-api/agents/add6443a-41bd-e414-f6ad-e58d267f4e95/displayicon.png',
 'agentPortraitUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/db/agents/jett_portrait.png',
 'countryCode': None,
 'platformInfo': {'platformSlug': 'riot',
  'platformUserId': None,
  'platformUserHandle': 'Hennessyy#AKA47',
  'platformUserIdentifier': 'Hennessyy#AKA47',
  'avatarUrl': None,
  'additionalParameters': None}}

In [26]:
player_summary[0]['stats'].keys()

dict_keys(['rank', 'currRank', 'score', 'scorePerRound', 'killsPerRound', 'kills', 'deaths', 'assists', 'kdRatio', 'damage', 'damagePerRound', 'singleKills', 'doubleKills', 'tripleKills', 'quadraKills', 'pentaKills', 'multiKills', 'grenadeCasts', 'ability1Casts', 'ability2Casts', 'ultimateCasts', 'grenadeCastsPerRound', 'ability1CastsPerRound', 'ability2CastsPerRound', 'ultimateCastsPerRound', 'plants', 'defuses', 'firstKills', 'firstDeaths', 'esr', 'firstKillsPerRound', 'firstDeathsPerRound', 'econRating', 'hsAccuracy', 'kast'])

In [27]:
player_summary[0]['stats']['rank']

{'rank': None,
 'percentile': None,
 'displayName': 'Rating',
 'displayCategory': None,
 'category': 'mmr',
 'metadata': {'iconUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/icons/tiersv2/0.png'},
 'value': 'Unranked',
 'displayValue': 'Unranked',
 'displayType': 'String'}

In [28]:
player_summary[0]['stats']['score']

{'rank': None,
 'percentile': None,
 'displayName': 'Score',
 'displayCategory': None,
 'category': None,
 'metadata': {},
 'value': 7621,
 'displayValue': '7,621',
 'displayType': 'Number'}

In [29]:
player_summary[0]['stats']['scorePerRound']

{'rank': None,
 'percentile': None,
 'displayName': 'Score per Round',
 'displayCategory': None,
 'category': None,
 'metadata': {},
 'value': 317.5416666666667,
 'displayValue': '318',
 'displayType': 'Number'}

## Win prediction data set creation

In [30]:
all_agents_list = ['Astra',
         'Breach',
         'Brimstone',
         'Chamber',
         'Cypher',
         'Fade',
         'Jett',
         'KAY/O',
         'Killjoy',
         'Neon',
         'Omen',
         'Phoenix',
         'Raze',
         'Reyna',
         'Sage',
         'Skye',
         'Sova',
         'Viper',
         'Yoru']

agent_roles = {'Astra':'Controller',
              'Breach':'Initiator',
         'Brimstone':'Controller',
         'Chamber':'Sentinel',
         'Cypher':'Sentinel',
         'Fade':'Initiator',
         'Jett':'Duelist',
         'KAY/O':'Initiator',
         'Killjoy':'Sentinel',
         'Neon':'Duelist',
         'Omen':'Controller',
         'Phoenix':'Duelist',
         'Raze':'Duelist',
         'Reyna':'Duelist',
         'Sage':'Sentinel',
         'Skye':'Initiator',
         'Sova':'Initiator',
         'Viper':'Controller',
         'Yoru':'Duelist'}
roles_list = ['Controller', 'Initiator', 'Duelist', 'Sentinel']
all_maps_list = ['Ascent', 
                 'Bind', 
                 'Breeze', 
                 'Fracture', 
                 'Haven', 
                 'Icebox', 
                 'Pearl', 
                 'Split']
weapons = ['Vandal',
             'Phantom',
             'Spectre',
             'Ghost',
             'Classic',
             'Operator',
             'Sheriff',
             'Guardian',
             'Marshal',
             'Odin',
             'Judge',
             'Bulldog',
             'Stinger',
             'Frenzy',
             'Ares',
             'Shorty',
             'Bucky']

In [31]:
invalid_keys = [
    'currRank',
]

def extract_player_rows(game_metadata, round_summary, player_summary, player_rounds_kills):
    winning_team = get_winner(round_summary)
    game_datetime = game_metadata['dateStarted']
    map_pick = game_metadata['mapName']
    
    data = list()
    
    for i in player_summary:
        new_record = dict()
        new_record['name'] = i['attributes']['platformUserIdentifier']
        
        agent = i['metadata']['agentName']
        agent_role = agent_roles[agent]
        
        new_record['game_datetime'] = game_datetime
        new_record['won_game'] = int(winning_team == i['metadata']['teamId'])
        new_record['team'] = i['metadata']['teamId']
        new_record['map_pick'] = map_pick
        
        for j in i['stats'].keys():
            if j in invalid_keys:
                continue
            new_record[j] = i['stats'][j]['value']
        
        for j in roles_list:
            if j == agent_role:
                new_record[f'role_{j}'] = 1
            else:
                new_record[f'role_{j}'] = 0
        
        for j in all_agents_list:
            if j == agent:
                new_record[f'agent_{j}'] = 1
            else:
                new_record[f'agent_{j}'] = 0
        
        weapon_kills_dict = {j: 0 for j in weapons}
        for j in player_rounds_kills:
            if 'platformInfo' in j and i['attributes']['platformUserIdentifier'] != j['platformInfo']['platformUserHandle']:
                if j['metadata']['weaponName'] in weapons:
                    weapon_kills_dict[j['metadata']['weaponName']] += 1
            
        data.append(new_record)
            
    return data
        
        
# processed_match_records = extract_player_rows(game_metadata, round_summary, player_summary, player_rounds_kills)
# processed_match_records[8]

In [32]:


def create_agent_row(agent_list, map_pick, game_win):

    
    for i in agent_list:
        if i not in all_agents_list:
            raise Exception(f'Invalid agent: {i}')
            
    if map_pick not in all_maps_list:
        raise Exception(f'Invalid map: {map_pick}')
    
    row = dict()
    
    roles_dict = {f'role_{i}': 0 for i in roles_list}
    
    for i in all_agents_list:
        row['agent_' + i] = int(i in agent_list)
                
    for i in agent_list:
        roles_dict[f'role_{agent_roles[i]}'] += 1
    
    roles_dict_keys = list(roles_dict.keys())
    
    for i in roles_dict_keys:
        for j in roles_dict_keys:
            if roles_list.index(i.split('_')[-1]) >= roles_list.index(j.split('_')[-1]):
                continue
            roles_dict[f'{i}*{j}'] = roles_dict[i] * roles_dict[j]
        
    for i in all_maps_list:
        row['map_' + i] = int(i == map_pick)
        
    row.update(roles_dict)
        
    row['game_win'] = game_win
    return row


def extract_team_rows(game_metadata, game_attributes, round_summary, player_summary):
    
    team_red_agents = list()
    team_blue_agents = list()
    
    map_pick = game_metadata['mapName']
        
    winning_team = get_winner(round_summary)
    
    for i in player_summary:
        if i['metadata']['teamId'] == 'Red':
            team_red_agents.append(i['metadata']['agentName'])
        if i['metadata']['teamId'] == 'Blue':
            team_blue_agents.append(i['metadata']['agentName'])
            
    row1 = create_agent_row(team_red_agents, map_pick, int('Red' == winning_team))
    row2 = create_agent_row(team_blue_agents, map_pick, int('Blue' == winning_team))
    
    return [row1, row2]
    
    




In [33]:
def get_all_processed_data(): 
    files = glob.glob(r'C:\Users\trist\OneDrive\Documents\game_data\valorant_raw/*.json')
    
    all_records = list()
    agent_records = list()
    
    for file in files:
        
        with open(file, 'r') as f:
            json_data = json.load(f)
        json_data = json.loads(json_data)
        if 'data' not in json_data:
            print(f'error {file}')
            continue
        game_metadata = json_data['data']['metadata']
        game_attributes = json_data['data']['attributes']

        player_rounds = [i for i in json_data['data']['segments'] if i['type'] == 'player-round']
        player_rounds_damage = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-damage']
        player_rounds_kills = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-kills']
        player_summary = [i for i in json_data['data']['segments'] if i['type'] == 'player-summary']
        round_summary = [i for i in json_data['data']['segments'] if i['type'] == 'round-summary']
        team_summary = [i for i in json_data['data']['segments'] if i['type'] == 'team-summary']
        
        all_records.extend(extract_player_rows(game_metadata, round_summary, player_summary, player_rounds_kills))
        agent_records.extend(extract_team_rows(game_metadata, game_attributes, round_summary, player_summary))
    return all_records, agent_records
        
all_records, all_agent_records = get_all_processed_data()
len(all_records), len(all_agent_records)


(31130, 6218)

In [34]:
user_record_count = dict()

for i in all_records:
    user_record_count.setdefault(i['name'], 0)
    user_record_count[i['name']] += 1

user_record_count_sorted = list()

for k, v in user_record_count.items():
    user_record_count_sorted.append({'user':k, 'count':v})

sorted(user_record_count_sorted, key = lambda x: x['count'], reverse = True)[:20]


[{'user': 'strawberry milk#우유갠디', 'count': 37},
 {'user': 'Thug Twerker#money', 'count': 30},
 {'user': 'CV WorstNub#2822', 'count': 28},
 {'user': 'tenz jr#333', 'count': 28},
 {'user': 'SmokedSalmon#4492', 'count': 28},
 {'user': 'IAmTempest#NA1', 'count': 26},
 {'user': 'Daddy Brim#OTP', 'count': 26},
 {'user': 'sussy#queer', 'count': 26},
 {'user': 'BishWhoDontMiss#6558', 'count': 26},
 {'user': '32Equr#6990', 'count': 25},
 {'user': 'YungCheddaBigD1k#2594', 'count': 25},
 {'user': 'Barbie#nails', 'count': 25},
 {'user': 'Tan#0008', 'count': 24},
 {'user': 'ZenitsuAG#7485', 'count': 24},
 {'user': 'I bark for mommy#6969', 'count': 24},
 {'user': 'CandyMan#7591', 'count': 23},
 {'user': 'DyslexicNepali#4122', 'count': 23},
 {'user': 'Arowana#queer', 'count': 23},
 {'user': 'DaddyMingze#NA1', 'count': 22},
 {'user': 'wakefull#icy', 'count': 22}]

In [35]:
all_records = [i for i in all_records if user_record_count[i['name']] >= 5]
len(all_records)

5828

In [36]:
all_records_df = pd.DataFrame.from_dict(all_records)
all_records_df

,name,game_datetime,won_game,team,map_pick,rank,score,scorePerRound,killsPerRound,kills,...,agent_Neon,agent_Omen,agent_Phoenix,agent_Raze,agent_Reyna,agent_Sage,agent_Skye,agent_Sova,agent_Viper,agent_Yoru
0,Yor#rice,2022-07-08T06:53:09.588+00:00,1,Red,Breeze,Unranked,5623,234.291667,0.833333,20,...,0,0,0,0,0,0,1,0,0,0
1,hardstuckiron#3570,2022-08-07T22:08:58.634+00:00,1,Blue,Icebox,Bronze 1,6042,402.800000,1.466667,22,...,0,0,0,0,0,0,0,0,0,0
2,Andreww#1178,2022-08-07T22:08:58.634+00:00,1,Blue,Icebox,Iron 3,2032,135.466667,0.466667,7,...,0,0,1,0,0,0,0,0,0,0
3,Boldzero#101,2022-07-27T05:21:43.67+00:00,1,Red,Ascent,Bronze 3,1052,210.400000,0.800000,4,...,0,1,0,0,0,0,0,0,0,0
4,Migegg#8765,2022-07-29T23:40:51.769+00:00,1,Red,Pearl,Bronze 2,4763,250.684211,0.842105,16,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5823,SmokedSalmon#4492,2022-07-27T04:07:20.866+00:00,1,Blue,Haven,Gold 2,4892,188.153846,0.615385,16,...,0,0,0,1,0,0,0,0,0,0
5824,Mrpancake#31798,2022-07-27T04:07:20.866+00:00,1,Blue,Haven,Silver 2,4089,157.269231,0.538462,14,...,0,0,0,0,0,0,0,0,0,0
5825,August#SUI,2022-07-27T04:07:20.866+00:00,1,Blue,Haven,Gold 1,7741,297.730769,1.038462,27,...,0,1,0,0,0,0,0,0,0,0
5826,Migegg#8765,2022-07-29T22:49:19.346+00:00,1,Blue,Icebox,Bronze 2,5645,256.590909,0.909091,20,...,0,0,0,0,0,0,0,0,0,0


In [37]:
all_records_df['name'].nunique()

411

In [38]:
all_records_df['game_datetime'].min()

'2022-03-26T16:46:26.008+00:00'

In [39]:
all_records_df['map_pick'].value_counts(normalize=True)

Fracture    0.148593
Ascent      0.147220
Breeze      0.143960
Haven       0.142759
Bind        0.141558
Pearl       0.135381
Icebox      0.132636
Split       0.007893
Name: map_pick, dtype: float64

In [40]:
all_records_df['killsPerRound'].describe()

count    5828.000000
mean        0.786707
std         1.247969
min         0.000000
25%         0.500000
50%         0.684211
75%         0.904762
max        39.000000
Name: killsPerRound, dtype: float64

In [41]:
all_records_df['rank'].value_counts(normalize=True)

Bronze 1      0.120110
Bronze 2      0.119423
Bronze 3      0.102265
Unranked      0.097461
Silver 1      0.094372
Silver 3      0.086136
Iron 3        0.070522
Silver 2      0.063830
Gold 1        0.052848
Gold 2        0.050618
Gold 3        0.043754
Iron 2        0.036033
Platinum 1    0.025051
Platinum 3    0.014070
Platinum 2    0.012011
Diamond 2     0.005148
Iron 1        0.004976
Diamond 1     0.001373
Name: rank, dtype: float64

In [42]:

def get_player_features(player_df):
    player_df = player_df.sort_values(by = ['game_datetime'])

    data = list()


    row_counter = 0
    for n, (idx, row) in enumerate(player_df.iterrows()):
        if n < 5:
            continue

        past_game = player_df.iloc[n-1:n].mean().to_dict()
        past_5_games = player_df.iloc[n-5:n].mean().to_dict()

        new_x = dict()

#         for k, v in past_game.items():
#             new_x[f'past_game_{k}'] = v

        for k, v in past_5_games.items():
            new_x[f'past_5_games_avg_{k}'] = v


        new_x['future_won_game'] =  row['won_game']
        data.append(new_x)


    data_df = pd.DataFrame.from_dict(data)
    data_df = data_df.fillna(0)
    return data_df



In [43]:

training_players, val_players = train_test_split(list(set(all_records_df['name'].tolist())))

training_data_dfs = list()
val_data_dfs = list()
all_data_dfs= list()

for i in training_players:
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    training_data_dfs.append(player_data_df)

    
for i in val_players:
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    val_data_dfs.append(player_data_df)
    
for i in list(set(all_records_df['name'].tolist())):
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    all_data_dfs.append(player_data_df)


    
training_data_df = pd.concat(training_data_dfs)
val_data_df = pd.concat(val_data_dfs)
all_data_df = pd.concat(all_data_dfs)

len(training_players), len(val_players)

(308, 103)

In [44]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# training_data_df.corr().sort_values(by=['future_won_game'])[['future_won_game']]

In [45]:
# training_data_df.isna().sum()

In [46]:
rf = RandomForestClassifier()

x = training_data_df.drop('future_won_game', axis = 1)
y = training_data_df['future_won_game']

x_val = val_data_df.drop('future_won_game', axis = 1)
y_val = val_data_df['future_won_game']

rf.fit(x, y)

preds = rf.predict(x_val)

accuracy_score(preds, y_val)

0.5248730964467005

In [47]:
x

,past_5_games_avg_won_game,past_5_games_avg_score,past_5_games_avg_scorePerRound,past_5_games_avg_killsPerRound,past_5_games_avg_kills,past_5_games_avg_deaths,past_5_games_avg_assists,past_5_games_avg_kdRatio,past_5_games_avg_damage,past_5_games_avg_damagePerRound,past_5_games_avg_singleKills,past_5_games_avg_doubleKills,past_5_games_avg_tripleKills,past_5_games_avg_quadraKills,past_5_games_avg_pentaKills,past_5_games_avg_multiKills,past_5_games_avg_grenadeCasts,past_5_games_avg_ability1Casts,past_5_games_avg_ability2Casts,past_5_games_avg_ultimateCasts,past_5_games_avg_grenadeCastsPerRound,past_5_games_avg_ability1CastsPerRound,past_5_games_avg_ability2CastsPerRound,past_5_games_avg_ultimateCastsPerRound,past_5_games_avg_plants,past_5_games_avg_defuses,past_5_games_avg_firstKills,past_5_games_avg_firstDeaths,past_5_games_avg_esr,past_5_games_avg_firstKillsPerRound,past_5_games_avg_firstDeathsPerRound,past_5_games_avg_econRating,past_5_games_avg_hsAccuracy,past_5_games_avg_kast,past_5_games_avg_role_Controller,past_5_games_avg_role_Initiator,past_5_games_avg_role_Duelist,past_5_games_avg_role_Sentinel,past_5_games_avg_agent_Astra,past_5_games_avg_agent_Breach,past_5_games_avg_agent_Brimstone,past_5_games_avg_agent_Chamber,past_5_games_avg_agent_Cypher,past_5_games_avg_agent_Fade,past_5_games_avg_agent_Jett,past_5_games_avg_agent_KAY/O,past_5_games_avg_agent_Killjoy,past_5_games_avg_agent_Neon,past_5_games_avg_agent_Omen,past_5_games_avg_agent_Phoenix,past_5_games_avg_agent_Raze,past_5_games_avg_agent_Reyna,past_5_games_avg_agent_Sage,past_5_games_avg_agent_Skye,past_5_games_avg_agent_Sova,past_5_games_avg_agent_Viper,past_5_games_avg_agent_Yoru
0,0.4,3620.0,191.834863,0.679204,12.8,14.6,4.4,1.155000,2350.4,123.703773,6.0,1.8,0.8,0.2,0.0,1.0,8.2,11.0,17.6,2.0,0.391533,0.577926,0.902733,0.103476,0.8,0.8,1.2,1.8,66.666667,0.066503,0.087671,58.0,10.726550,63.746286,0.4,0.4,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0
1,0.4,3478.8,166.787352,0.567439,11.8,18.0,4.8,0.668043,2341.6,111.066669,7.2,1.4,0.6,0.0,0.0,0.6,9.6,12.2,21.2,2.4,0.433162,0.583356,0.980109,0.114788,1.2,1.2,1.6,2.8,35.555556,0.073743,0.126132,44.4,12.358918,62.298323,0.2,0.6,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0
2,0.6,3881.4,183.292095,0.616056,13.0,17.4,4.2,0.786313,2581.0,120.741768,6.8,2.2,0.6,0.0,0.0,0.6,15.2,12.2,19.0,2.6,0.674664,0.577823,0.876551,0.122693,1.4,1.4,1.8,2.4,42.222222,0.082439,0.107160,48.0,10.821637,62.654054,0.2,0.4,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0
3,0.6,3500.2,179.399285,0.585990,11.6,16.0,4.0,0.751313,2369.4,119.390134,6.6,2.2,0.2,0.0,0.0,0.2,16.2,11.8,16.4,2.4,0.869435,0.617038,0.765440,0.121386,1.2,1.2,1.6,2.0,52.222222,0.081132,0.083631,49.8,11.154318,77.686734,0.2,0.4,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0
4,0.4,4031.0,208.840951,0.666943,13.0,15.8,4.4,0.831313,2660.6,136.035372,5.8,2.6,0.4,0.2,0.0,0.6,15.8,12.4,15.0,2.6,0.876578,0.652753,0.722583,0.133291,1.0,1.0,1.6,2.2,50.555556,0.082322,0.095535,57.2,10.348347,77.924829,0.0,0.4,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,0.8,3571.8,199.710763,0.736355,13.0,10.0,8.0,1.701255,2322.0,131.446806,6.8,1.6,1.0,0.0,0.0,1.0,4.4,2.4,8.2,1.8,0.258345,0.141328,0.471002,0.091525,3.6,3.6,1.0,0.4,45.000000,0.049037,0.020856,44.2,8.664086,84.596116,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
11,0.8,3297.0,183.124973,0.645829,11.4,10.8,6.6,1.497619,2240.6,125.641016,6.4,1.6,0.6,0.0,0.0,0.6,4.6,2.8,8.2,1.2,0.265187,0.160802,0.466791,0.059947,4.0,4.0,1.0,0.8,45.000000,0.049037,0.040856,44.6,9.079670,81.7540

In [48]:
fi_list = list()
for i, j in zip(x_val.columns, rf.feature_importances_):
    fi_list.append({'column':i, 'importance':j})
    
pd.DataFrame.from_dict(fi_list).sort_values(by = ['importance'])[-10:]

,column,importance
28,past_5_games_avg_esr,0.031694
29,past_5_games_avg_firstKillsPerRound,0.031841
31,past_5_games_avg_econRating,0.031889
21,past_5_games_avg_ability1CastsPerRound,0.032306
7,past_5_games_avg_kdRatio,0.032367
22,past_5_games_avg_ability2CastsPerRound,0.032939
16,past_5_games_avg_grenadeCasts,0.033001
32,past_5_games_avg_hsAccuracy,0.033567
33,past_5_games_avg_kast,0.033672
20,past_5_games_avg_grenadeCastsPerRound,0.035758


In [49]:
rf = LogisticRegression()

x = training_data_df.drop('future_won_game', axis = 1)
y = training_data_df['future_won_game']

x_val = val_data_df.drop('future_won_game', axis = 1)
y_val = val_data_df['future_won_game']

rf.fit(x, y)

preds = rf.predict(x_val)

accuracy_score(preds, y_val)

C:\Users\trist\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5015228426395939

In [50]:

df_interaction = pd.DataFrame()

df_interaction['future_won_game'] = all_data_df['future_won_game']

columns_list = training_data_df.columns.tolist()

for i in columns_list:
    df_interaction[i] =  all_data_df[i]
    for j in columns_list:
        if  i == 'future_won_game' or j == 'future_won_game':
            continue
        if columns_list.index(i) >=columns_list.index(j):
            continue
        df_interaction[f'{i}_mul_{j}'] = all_data_df[i]*all_data_df[j]
        
df_interaction.head()



,future_won_game,past_5_games_avg_won_game,past_5_games_avg_won_game_mul_past_5_games_avg_score,past_5_games_avg_won_game_mul_past_5_games_avg_scorePerRound,past_5_games_avg_won_game_mul_past_5_games_avg_killsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_kills,past_5_games_avg_won_game_mul_past_5_games_avg_deaths,past_5_games_avg_won_game_mul_past_5_games_avg_assists,past_5_games_avg_won_game_mul_past_5_games_avg_kdRatio,past_5_games_avg_won_game_mul_past_5_games_avg_damage,past_5_games_avg_won_game_mul_past_5_games_avg_damagePerRound,past_5_games_avg_won_game_mul_past_5_games_avg_singleKills,past_5_games_avg_won_game_mul_past_5_games_avg_doubleKills,past_5_games_avg_won_game_mul_past_5_games_avg_tripleKills,past_5_games_avg_won_game_mul_past_5_games_avg_quadraKills,past_5_games_avg_won_game_mul_past_5_games_avg_pentaKills,past_5_games_avg_won_game_mul_past_5_games_avg_multiKills,past_5_games_avg_won_game_mul_past_5_games_avg_grenadeCasts,past_5_games_avg_won_game_mul_past_5_games_avg_ability1Casts,past_5_games_avg_won_game_mul_past_5_games_avg_ability2Casts,past_5_games_avg_won_game_mul_past_5_games_avg_ultimateCasts,past_5_games_avg_won_game_mul_past_5_games_avg_grenadeCastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ability1CastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ability2CastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ultimateCastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_plants,past_5_games_avg_won_game_mul_past_5_games_avg_defuses,past_5_games_avg_won_game_mul_past_5_games_avg_firstKills,past_5_games_avg_won_game_mul_past_5_games_avg_firstDeaths,past_5_games_avg_won_game_mul_past_5_games_avg_esr,past_5_games_avg_won_game_mul_past_5_games_avg_firstKillsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_firstDeathsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_econRating,past_5_games_avg_won_game_mul_past_5_games_avg_hsAccuracy,past_5_games_avg_won_game_mul_past_5_games_avg_kast,past_5_games_avg_won_game_mul_past_5_games_avg_role_Controller,past_5_games_avg_won_game_mul_past_5_games_avg_role_Initiator,past_5_games_avg_won_game_mul_past_5_games_avg_role_Duelist,past_5_games_avg_won_game_mul_past_5_games_avg_role_Sentinel,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Astra,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Breach,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Brimstone,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Chamber,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Cypher,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Fade,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Jett,past_5_games_avg_won_game_mul_past_5_games_avg_agent_KAY/O,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Killjoy,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Neon,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Omen,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Phoenix,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Raze,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Reyna,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Sage,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Skye,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Sova,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Viper,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Yoru,past_5_games_avg_score,past_5_games_avg_score_mul_past_5_games_avg_scorePerRound,past_5_games_avg_score_mul_past_5_games_avg_killsPerRound,past_5_games_avg_score_mul_past_5_games_avg_kills,past_5_games_avg_score_mul_past_5_games_avg_deaths,past_5_games_avg_score_mul_past_5_games_avg_assists,past_5_games_avg_score_mul_past_5_games_avg_kdRatio,past_5_games_avg_score_mul_past_5_games_avg_damage,past_5_games_avg_score_mul_past_5_games_avg_damagePerRound,past_5_games_avg_score_mul_past_5_games_avg_singleKills,past_5_games_avg_score_mul_past_5_games_avg_doubleKills,past_5_games_avg_score_mul_past_5_games_avg_tripleKills,past_5_ga

In [51]:
pd.set_option('max_colwidth', 400)



fi_list = list()

for i in df_interaction.columns:
    
    if i == 'future_won_game':
        continue
        
    rel = linregress(df_interaction['future_won_game'], df_interaction[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False]).head(20)

,column,slope,intercept,rvalue,r2,pvalue,stderr
352,past_5_games_avg_assists_mul_past_5_games_avg_econRating,18.537801,273.679792,0.077334,0.005981,0.000002,3.891852
379,past_5_games_avg_kdRatio_mul_past_5_games_avg_damage,337.032818,3151.782993,0.073449,0.005395,0.000006,74.521506
382,past_5_games_avg_kdRatio_mul_past_5_games_avg_doubleKills,0.336538,2.975626,0.072705,0.005286,0.000008,0.075177
451,past_5_games_avg_damage_mul_past_5_games_avg_econRating,14171.100182,165617.445917,0.072415,0.005244,0.000008,3178.356375
453,past_5_games_avg_damage_mul_past_5_games_avg_kast,10279.611497,193112.890527,0.072264,0.005222,0.000009,2310.411975
592,past_5_games_avg_doubleKills_mul_past_5_games_avg_econRating,14.500620,154.545813,0.072177,0.005210,0.000009,3.263062
330,past_5_games_avg_assists_mul_past_5_games_avg_damagePerRound,39.407973,642.556708,0.072048,0.005191,0.000009,8.883955
428,past_5_games_avg_damage,116.289608,2694.958854,0.071729,0.005145,0.000010,26.332744
328,past_5_games_avg_assists_mul_past_5_games_avg_kdRatio,0.397843,5.174789,0.071294,0.005083,0.000012,0.090640
402,past_5_games_avg_kdRatio_mul_past_5_games_avg_econRating,8.863000,68.860947,0.071229,0.005074,0.000012,2.021114


In [52]:
fi_list = list()

for i in all_data_df.columns:
    if i == 'future_won_game':
        continue
    rel = linregress(all_data_df['future_won_game'], all_data_df[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False])

,column,slope,intercept,rvalue,r2,pvalue,stderr
8,past_5_games_avg_damage,116.289608,2694.958854,0.071729,5.145088e-03,0.000010,26.332744
9,past_5_games_avg_damagePerRound,5.097778,133.937582,0.069947,4.892584e-03,0.000017,1.183909
7,past_5_games_avg_kdRatio,0.068529,1.088021,0.067844,4.602750e-03,0.000030,0.016411
31,past_5_games_avg_econRating,2.313922,57.828333,0.063781,4.068079e-03,0.000088,0.589577
11,past_5_games_avg_doubleKills,0.126216,2.499688,0.062011,3.845334e-03,0.000138,0.033081
2,past_5_games_avg_scorePerRound,-37.443960,249.522948,-0.054121,2.929090e-03,0.000882,11.249939
3,past_5_games_avg_killsPerRound,-0.116565,0.845654,-0.053035,2.812663e-03,0.001119,0.035741
1,past_5_games_avg_score,128.315733,4156.553021,0.051324,2.634132e-03,0.001613,40.659322
28,past_5_games_avg_esr,4.281735,60.363681,0.049810,2.481015e-03,0.002210,1.398097
4,past_5_games_avg_kills,0.439078,14.265833,0.048604,2.362338e-03,0.002824,0.146936



## Agent pick prediction data set creation

In [53]:
all_records, all_agent_records = get_all_processed_data()


In [54]:
# agents = list()

# for i in all_records:
#     agents.append(i['agent'])
    
# sorted(list(set(agents)))

In [55]:
maps = list()

for i in all_records:
    maps.append(i['map_pick'])
    
sorted(list(set(maps)))

['Ascent', 'Bind', 'Breeze', 'Fracture', 'Haven', 'Icebox', 'Pearl', 'Split']

In [56]:
all_agent_records_df = pd.DataFrame.from_dict(all_agent_records)
all_agent_records_df.head()

,agent_Astra,agent_Breach,agent_Brimstone,agent_Chamber,agent_Cypher,agent_Fade,agent_Jett,agent_KAY/O,agent_Killjoy,agent_Neon,agent_Omen,agent_Phoenix,agent_Raze,agent_Reyna,agent_Sage,agent_Skye,agent_Sova,agent_Viper,agent_Yoru,map_Ascent,map_Bind,map_Breeze,map_Fracture,map_Haven,map_Icebox,map_Pearl,map_Split,role_Controller,role_Initiator,role_Duelist,role_Sentinel,role_Controller*role_Initiator,role_Controller*role_Duelist,role_Controller*role_Sentinel,role_Initiator*role_Duelist,role_Initiator*role_Sentinel,role_Duelist*role_Sentinel,game_win
0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,2,1,1,2,1,2,1,2,1
1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,2,1,1,2,1,2,1,2,0
2,0,0,0,1,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2,2,0,0,0,2,2,4,0
3,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,3,0,0,0,0,0,6,1
4,0,0,0,1,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,2,2,0,2,2,0,0,4,1


In [57]:
train_df, val_df = train_test_split(all_agent_records_df)

train_x = train_df.drop('game_win', axis = 1)
train_y = train_df['game_win']

val_x = val_df.drop('game_win', axis = 1)
val_y = val_df['game_win']
train_x.head()

,agent_Astra,agent_Breach,agent_Brimstone,agent_Chamber,agent_Cypher,agent_Fade,agent_Jett,agent_KAY/O,agent_Killjoy,agent_Neon,agent_Omen,agent_Phoenix,agent_Raze,agent_Reyna,agent_Sage,agent_Skye,agent_Sova,agent_Viper,agent_Yoru,map_Ascent,map_Bind,map_Breeze,map_Fracture,map_Haven,map_Icebox,map_Pearl,map_Split,role_Controller,role_Initiator,role_Duelist,role_Sentinel,role_Controller*role_Initiator,role_Controller*role_Duelist,role_Controller*role_Sentinel,role_Initiator*role_Duelist,role_Initiator*role_Sentinel,role_Duelist*role_Sentinel
4737,0,0,0,1,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,2,1,1,2,1,2,1,2
2881,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,2,2,0,2,2,0,0,4
2966,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,2,1,1,2,1,1,2,2,1
4935,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,3,1,0,3,1,0,0,3
4290,0,0,1,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,2,0,2,1,0,4,2,0,0,2


In [58]:
rf = RandomForestClassifier()

rf.fit(train_x, train_y)

preds = rf.predict(val_x)

accuracy_score(preds, val_y)

0.5151125401929261

In [59]:
fi_list = list()
for i, j in zip(train_x.columns, rf.feature_importances_):
    fi_list.append({'column':i, 'importance':j})
    
pd.DataFrame.from_dict(fi_list).sort_values(by = ['importance'])[-10:]

,column,importance
22,map_Fracture,0.033799
19,map_Ascent,0.033929
14,agent_Sage,0.034710
18,agent_Yoru,0.036175
9,agent_Neon,0.037380
3,agent_Chamber,0.037615
11,agent_Phoenix,0.039944
12,agent_Raze,0.047323
13,agent_Reyna,0.049313
6,agent_Jett,0.050840


In [60]:
fi_list = list()

for i in train_x.columns:
    rel = linregress(train_y, train_x[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False])

,column,slope,intercept,rvalue,r2,pvalue,stderr
17,agent_Viper,0.030680,0.263539,0.034209,1.170277e-03,0.019487,0.013128
10,agent_Omen,-0.027722,0.360768,-0.029118,8.478817e-04,0.046782,0.013939
9,agent_Neon,-0.023391,0.214072,-0.029106,8.471317e-04,0.046879,0.011767
36,role_Duelist*role_Sentinel,0.071211,2.224733,0.027587,7.610440e-04,0.059610,0.037795
30,role_Sentinel,0.036933,1.281876,0.027441,7.529978e-04,0.060975,0.019707
13,agent_Reyna,0.026560,0.576546,0.026998,7.289102e-04,0.065263,0.014404
5,agent_Fade,0.021297,0.198721,0.026174,6.850982e-04,0.073909,0.011914
3,agent_Chamber,0.024862,0.430704,0.025024,6.261946e-04,0.087526,0.014548
7,agent_KAY/O,-0.019503,0.214499,-0.024164,5.838847e-04,0.098975,0.011819
2,agent_Brimstone,0.019675,0.207676,0.023847,5.687028e-04,0.103472,0.012081


In [61]:
def create_interactions(df: pd.DataFrame) -> pd.DataFrame:
    df_interaction = pd.DataFrame()

    df_interaction['game_win'] = df['game_win']

    columns_list = sorted(df.columns.tolist())

    for i in columns_list:
        df_interaction[i] = df[i]
        for j in columns_list:
            if  i == 'game_win' or j == 'game_win':
                continue
            if columns_list.index(i) >=columns_list.index(j):
                continue
            df_interaction[f'{i}_mul_{j}'] = df[i]*df[j]
    df_interaction.index = df.index
    return df_interaction

In [62]:
train_df, val_df = train_test_split(all_agent_records_df)


In [63]:
fi_list = list()

df_interaction_train = create_interactions(train_df)

for i in df_interaction_train.columns:
    if i == 'game_win':
        continue
    rel = linregress(df_interaction_train['game_win'], df_interaction_train[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False]).head(50)

,column,slope,intercept,rvalue,r2,pvalue,stderr
688,role_Duelist*role_Sentinel,0.125122,2.186729,0.048653,0.002367,0.000889,0.037624
687,role_Duelist_mul_role_Sentinel,0.125122,2.186729,0.048653,0.002367,0.000889,0.037624
422,agent_Reyna_mul_role_Duelist*role_Sentinel,0.147955,1.335177,0.047984,0.002302,0.001047,0.045112
118,agent_Chamber_mul_agent_Reyna,0.041569,0.233518,0.047738,0.002279,0.001111,0.012740
702,role_Sentinel,0.059444,1.267546,0.043794,0.001918,0.002779,0.019863
136,agent_Chamber_mul_role_Duelist,0.082628,0.718843,0.043525,0.001894,0.002951,0.027780
426,agent_Reyna_mul_role_Sentinel,0.064779,0.712037,0.040489,0.001639,0.005688,0.023415
683,role_Duelist_mul_role_Duelist*role_Sentinel,0.274926,4.298171,0.040183,0.001615,0.006064,0.100134
321,agent_Neon_mul_role_Initiator,-0.034494,0.185878,-0.038615,0.001491,0.008361,0.013075
306,agent_Neon_mul_agent_Yoru,-0.012813,0.035304,-0.038207,0.001460,0.009074,0.004908


In [ ]:
import random
results = list()

counter = 0

while True:
    counter += 1
    if counter > 2000:
        break
    i = random.randint(10, 100)
    j = random.randint(10, 99) * .01
    k = random.randint(1, 100)
    max_depth = random.randint(2, 10)
    

    model = RandomForestClassifier(n_estimators=i, max_features = j, max_depth=max_depth)
    pca = PCA(n_components=k)

    df_train_int = create_interactions(train_df)
    df_val_int = create_interactions(val_df)

    x_train_pca = pca.fit_transform(df_train_int.drop('game_win', axis = 1))
    x_val_pca = pca.transform(df_val_int.drop('game_win', axis = 1))
    
    x_train_pca_df = pd.DataFrame(columns = [f'pca_{col}' for col in range(k)],
                                 data = x_train_pca, index = train_df.index)
    x_val_pca_df = pd.DataFrame(columns = [f'pca_{col}' for col in range(k)],
                             data = x_val_pca, index = val_df.index)
    
    df_concat_train = pd.concat([x_train_pca_df, train_df.drop('game_win', axis=1)], axis=1)
    df_concat_val = pd.concat([x_val_pca_df, val_df.drop('game_win', axis=1)], axis=1)

    model.fit(df_concat_train, train_df['game_win'])

    preds = model.predict(df_concat_val)
    score= accuracy_score(preds, val_df['game_win'])

    results.append({'n_estimators':i,
                   'max_features':j,
                   'n_components':k,
                    'max_depth':max_depth,
                   'score':score})
    if score >= sorted(results, key = lambda x: x['score'], reverse = True)[0]['score'] or counter%40 == 0:
        print(counter, i, j, k, score, sorted(results, key = lambda x: x['score'], reverse = True)[:3])
        
            
sorted(results, key = lambda x: x['score'], reverse = True)[:10]

1 49 0.59 30 0.5299035369774919 [{'n_estimators': 49, 'max_features': 0.59, 'n_components': 30, 'max_depth': 4, 'score': 0.5299035369774919}]
5 98 0.65 48 0.5305466237942122 [{'n_estimators': 98, 'max_features': 0.65, 'n_components': 48, 'max_depth': 8, 'score': 0.5305466237942122}, {'n_estimators': 49, 'max_features': 0.59, 'n_components': 30, 'max_depth': 4, 'score': 0.5299035369774919}, {'n_estimators': 85, 'max_features': 0.24, 'n_components': 67, 'max_depth': 9, 'score': 0.5176848874598071}]
7 46 0.9 51 0.5466237942122186 [{'n_estimators': 46, 'max_features': 0.9, 'n_components': 51, 'max_depth': 3, 'score': 0.5466237942122186}, {'n_estimators': 98, 'max_features': 0.65, 'n_components': 48, 'max_depth': 8, 'score': 0.5305466237942122}, {'n_estimators': 49, 'max_features': 0.59, 'n_components': 30, 'max_depth': 4, 'score': 0.5299035369774919}]
40 20 0.18 6 0.5106109324758843 [{'n_estimators': 46, 'max_features': 0.9, 'n_components': 51, 'max_depth': 3, 'score': 0.5466237942122186},

In [ ]:
x_train_pca_df.shape, train_df.shape , 

In [ ]:
# sorted(results, key = lambda x: x['score'], reverse = True)

In [ ]:
from sklearn.linear_model import LogisticRegression

train_df, val_df = train_test_split(all_agent_records_df, random_state = 42)
n_components = 93
n_estimators = 48
max_features = 0.42
max_depth = 6

model = RandomForestClassifier(n_estimators=n_estimators, max_features = max_features, max_depth=max_depth)
pca = PCA(n_components=n_components)

df_train_int = create_interactions(train_df)
df_val_int = create_interactions(val_df)

x_train_pca = pca.fit_transform(df_train_int.drop('game_win', axis = 1))
x_val_pca = pca.transform(df_val_int.drop('game_win', axis = 1))

x_train_pca_df = pd.DataFrame(columns = [f'pca_{col}' for col in range(n_components)],
                                 data = x_train_pca, index = train_df.index)
x_val_pca_df = pd.DataFrame(columns = [f'pca_{col}' for col in range(n_components)],
                         data = x_val_pca, index = val_df.index)

df_concat_train = pd.concat([x_train_pca_df, train_df.drop('game_win', axis=1)], axis=1)
df_concat_val = pd.concat([x_val_pca_df, val_df.drop('game_win', axis=1)], axis=1)



model.fit(df_concat_train, train_df['game_win'])

preds = model.predict(df_concat_val)
accuracy_score(preds, val_df['game_win'])


In [ ]:
fi_list = list()
for i, j in zip(df_concat_train.columns, model.feature_importances_):
    fi_list.append({'column':i, 'importance':j})
    
pd.DataFrame.from_dict(fi_list).sort_values(by = ['importance'])[-10:]

In [ ]:
fi_list = list()


fi_df = pd.concat([x_train_pca_df, train_df], axis=1)

for i in fi_df.columns:
    if i == 'game_win':
        continue
    rel = linregress(fi_df['game_win'], fi_df[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False]).head(50)

In [ ]:
fi_df.corr()[[i for i in fi_df.columns if 'pca' not in i]]

In [ ]:

model = RandomForestClassifier(n_estimators=n_estimators, max_features = max_features, max_depth=max_depth)
pca = PCA(n_components=n_components)

df_all = create_interactions(all_agent_records_df)

x_pca = pca.fit_transform(df_all.drop('game_win', axis = 1))
x_pca_df = pd.DataFrame(columns = [f'pca_{col}' for col in range(n_components)],
                                 data = x_pca, index = df_all.index)
df_concat = pd.concat([x_pca_df, all_agent_records_df.drop('game_win', axis=1)], axis=1)

model.fit(df_concat, all_agent_records_df['game_win'])

preds = model.predict(df_concat)
preds.shape, df_concat.shape, df_all.shape

In [ ]:
def predict_best_lineup(model, vectorizer, map_pick, current_agent_list):
    
    agent_score = list()
    
    agent_tuples = list()
    
    for i1 in all_agents_list:
        for i2 in all_agents_list:
            for i3 in all_agents_list:
                for i4 in all_agents_list:
                    for i5 in all_agents_list:
                        if len(set([i1, i2, i3, i4, i5])) < 5:
                            continue
                        else:
                            agent_tuples.append(tuple(sorted([i1, i2, i3, i4, i5])))
    agent_tuples = list(set(agent_tuples))
    matched_agent_tuples = list()
    
    for i in agent_tuples:
        match = True
        for j in current_agent_list:
            if j not in i:
                match = False
        if match:
            matched_agent_tuples.append(i)
    
    inputs = list()
    features = list()
    
    for i in matched_agent_tuples:
        inputs.append({'agent1_name': i[0], 'agent2_name': i[1], 'agent3_name': i[2], 'agent4_name': i[3], 'agent5_name': i[4]})
        features.append(create_agent_row(list(i), map_pick, None))
    
    features_df = pd.DataFrame.from_dict(features)
    features_df_interactions = create_interactions(features_df)
    features_df_interactions_pca = pca.transform(features_df_interactions.drop('game_win', axis = 1))
    features_df_interactions_pca_df = pd.DataFrame(columns = [f'pca_{col}' for col in range(n_components)],
                                 data = features_df_interactions_pca, index = features_df.index)
    df_concat = pd.concat([features_df_interactions_pca_df, features_df.drop('game_win', axis=1)], axis=1)

    inputs_df = pd.DataFrame.from_dict(inputs)
    
    inputs_df.index = features_df.index

    inputs_df['win_prob'] = model.predict_proba(df_concat)[:,-1]
    return inputs_df.sort_values('win_prob', ascending = False)


map_pick = 'Icebox'
current_agent_list = ['Phoenix',
                     'Reyna',
                      'Raze',
                     ]

predict_best_lineup(model, pca, map_pick, current_agent_list)

In [ ]:
['Ascent', 'Bind', 'Breeze', 'Fracture', 'Haven', 'Icebox', 'Pearl', 'Split']

In [ ]:
map_pick = 'Ascent'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Bind'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Breeze'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Fracture'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Haven'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Icebox'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Pearl'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Split'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()